In [4]:
# Imports

%reload_ext autoreload
%autoreload 3

import numpy as np
import torch
from torch import nn
import pytorch_lightning as pl

from src.cogponder import CogPonderModel
from src.cogponder.datasets import CogPonderDataModule
from src.cogponder.datasets.stroop_sro import StroopSRODataset

In [5]:
# Experiment

step_duration = 10  # in ms
batch_size = 512

dataset = StroopSRODataset(response_step_interval=step_duration)

datamodule = CogPonderDataModule(dataset, batch_size=batch_size, num_workers=8)

configs = {
    'inputs_dim': dataset._data['stimuli'].shape[1],
    'outputs_dim': np.unique(dataset._data['responses'].data).shape[0],
    'embeddings_dim': 10,
    'response_loss_beta': 1.,
    'time_loss_beta': 10.,
    'learning_rate': 1e-2,
    'max_response_step': dataset._data['response_steps'].data.max() + 10,
    'n_contexts': np.unique(dataset._data['contexts'].data).shape[0],
    'n_subjects': np.unique(dataset._data['subject_ids'].data).shape[0],
    'subject_embeddings_dim': 2,
    'task': 'nback'
}

model = CogPonderModel(**configs)

trainer = pl.Trainer(max_epochs=1000, accelerator='cpu', log_every_n_steps=2)

trainer.fit(model, datamodule=datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name               | Type                 | Params
------------------------------------------------------------
0 | operator_node      | SimpleOperatorModule | 124   
1 | halt_node          | HaltingModule        | 121   
2 | recurrence_node    | RecurrenceModule     | 480   
3 | subject_embeddings | Embedding            | 1.0 K 
4 | embeddings         | Embedding            | 20    
5 | resp_loss_fn       | ResponseLoss         | 0     
6 | time_loss_fn       | ResponseTimeLoss     | 0     
------------------------------------------------------------
1.8 K     Trainable params
0         Non-trainable params
1.8 K     Total params
0.007     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]